In [259]:
df_raw = spark.read.options(header='False',wholetext=True)\
    .text('adl://cp-bizops-c15.azuredatalakestore.net/local/users/lowen/UIUC/dblp.xml')
#df_raw['value_1']=df_raw['value'].apply(lambda x: ' '.join(x) )

#df_text=' '.join(df_raw['value_1'])
#df_raw.write.mode("overwrite").saveAsTable("default.dblp")
df_raw.show()

StatementMeta(sparkpool3, 22, 3, Finished, Available)

+--------------------+
|               value|
+--------------------+
|<?xml version="1....|
|<!DOCTYPE dblp SY...|
|              <dblp>|
|<phdthesis mdate=...|
|<author>Carmen He...|
|<title>Modell zur...|
|   <year>2010</year>|
|<school>Aarhus Un...|
|<pages>1-315</pages>|
|<isbn>978-3-86596...|
|<ee>http://d-nb.i...|
|</phdthesis><phdt...|
|<author>Gerd Hoff...|
|<title>Ein Verfah...|
|   <year>2002</year>|
|<school>Universit...|
|<ee>http://ubt.op...|
|<ee>https://nbn-r...|
|<ee>http://d-nb.i...|
|        </phdthesis>|
+--------------------+
only showing top 20 rows

In [260]:
df=df_raw.select('value').collect()

StatementMeta(sparkpool3, 22, 4, Finished, Available)

In [261]:
mstr=""
flag=False
dblp=[]
for row in df:
    s=str(row.value)
    if s.__contains__('</phdthesis>'):
        i=s.index('</phdthesis>')
        s1=s[0:i+len('</phdthesis>')]
        mstr=mstr+s1
        dblp.append(mstr)
        mstr=""
        flag=False
    if s.__contains__('<phdthesis'):
        i=s.index('<phdthesis')
        mstr=mstr+s[i:]
        flag=True
    else:
        if flag==True:
            mstr=mstr+s

StatementMeta(sparkpool3, 22, 5, Finished, Available)

In [262]:
import xml.etree.ElementTree as ET 
list_dblp=[]
for d in dblp:
    dict_dblp={}
    try:
        tree = ET.fromstring(d.replace('&','')) 
        for child in tree:
            #print(child.tag, ' '.join(child.itertext()))
            dict_dblp[child.tag]=' '.join(child.itertext())
        list_dblp.append(dict_dblp)
    except :
        print(d)

list_dblp[:2]

StatementMeta(sparkpool3, 22, 6, Finished, Available)

[{'author': 'Carmen Heine', 'title': 'Modell zur Produktion von Online-Hilfen.', 'year': '2010', 'school': 'Aarhus University', 'pages': '1-315', 'isbn': '978-3-86596-263-8', 'ee': 'http://d-nb.info/996064095'}, {'author': 'Gerd Hoff', 'title': 'Ein Verfahren zur thematisch spezialisierten Suche im Web und seine Realisierung im Prototypen HomePageSearch', 'year': '2002', 'school': 'University of Trier, Germany', 'ee': 'http://d-nb.info/971713243'}]

In [263]:
import xml.etree.ElementTree as ET 
list_dblp_together=[]
for d in dblp:
    dict_dblp={}
    try:
        tree = ET.fromstring(d.replace('&','')) 
        #dict_dblp[child.tag]=' '.join(child.itertext())
        list_dblp_together.append(' '.join(tree.itertext()))
    except :
        print(d)

list_dblp_together[:2]

StatementMeta(sparkpool3, 22, 7, Finished, Available)

['Carmen Heine Modell zur Produktion von Online-Hilfen. 2010 Aarhus University 1-315 978-3-86596-263-8 http://d-nb.info/996064095', 'Gerd Hoff Ein Verfahren zur thematisch spezialisierten Suche im Web und seine Realisierung im Prototypen HomePageSearch 2002 University of Trier, Germany http://ubt.opus.hbz-nrw.de/volltexte/2004/146/ https://nbn-resolving.org/urn:nbn:de:hbz:385-1468 http://d-nb.info/971713243']

In [264]:
columns = ["value"]

df_all = spark.createDataFrame(data=[[p] for p in list_dblp_together],schema =columns)
df_all.coalesce(1).write.format("text").option("header", "false").mode("overwrite").save('adl://cp-bizops-c15.azuredatalakestore.net/local/users/lowen/UIUC/dblp_all.txt')

df_sample = spark.createDataFrame(data=[[p] for p in list_dblp_together[:500]],schema =columns)
df_sample.coalesce(1).write.format("text").option("header", "false").mode("overwrite").save('adl://cp-bizops-c15.azuredatalakestore.net/local/users/lowen/UIUC/dblp_sample.txt')

print(len(list_dblp),len(list_dblp_together),df_all.count(),df_sample.count())

StatementMeta(sparkpool3, 22, 8, Finished, Available)

78704 78704 78704 500

In [265]:
import gensim
from gensim import corpora
from pprint import pprint
###1. get the word list

list_dblp_together[:2]

StatementMeta(sparkpool3, 22, 9, Finished, Available)

['Carmen Heine Modell zur Produktion von Online-Hilfen. 2010 Aarhus University 1-315 978-3-86596-263-8 http://d-nb.info/996064095', 'Gerd Hoff Ein Verfahren zur thematisch spezialisierten Suche im Web und seine Realisierung im Prototypen HomePageSearch 2002 University of Trier, Germany http://ubt.opus.hbz-nrw.de/volltexte/2004/146/ https://nbn-resolving.org/urn:nbn:de:hbz:385-1468 http://d-nb.info/971713243']

In [266]:
texts = [doc.split() for doc in list_dblp_together]

texts[:2]
dictionary = corpora.Dictionary(texts)
print(dictionary)

StatementMeta(sparkpool3, 22, 10, Finished, Available)

Dictionary(358113 unique tokens: ['1-315', '2010', '978-3-86596-263-8', 'Aarhus', 'Carmen']...)

In [267]:
from gensim.utils import simple_preprocess
tokenized_list = [simple_preprocess(doc) for doc in list_dblp_together]
##remove words with length<=3
tokenized_list=[[w for w in s if len(w)>3]  for s in tokenized_list ]
##stop words
stopword=['http','https','university', 'info']
tokenized_list=[[w for w in s if w not in stopword]  for s in tokenized_list ]

dct = corpora.Dictionary(tokenized_list)

#corpus = [dct.doc2bow(line) for line in tokenized_list]
#corpus[:1]

dct

StatementMeta(sparkpool3, 22, 11, Finished, Available)

In [268]:
###1. get the dictionary for all data
from collections import Counter
##1.1. get the (word, count) pair dictionary
dictionary={}

for s in tokenized_list:
    for w in s:
        dictionary[w]=dictionary.get(w,0)+1
##1.2. sorting to get the top 500 words
top_words=Counter(dictionary)
top_words=top_words.most_common(200)
#topwords_dict
top_words=[w[0] for w in top_words]
###2. deal with the two words phrase


StatementMeta(sparkpool3, 22, 12, Finished, Available)

In [269]:
word_dict={}
i=1
for word in top_words:
    word_dict[word]=i
    i=i+1
#print(word_dict.get('germany'))
word_dict

StatementMeta(sparkpool3, 22, 13, Finished, Available)

{'ethos': 1, 'search': 2, 'base': 3, 'germany': 4, 'ndltd': 5, 'handle': 6, 'archives': 7, 'ouvertes': 8, 'library': 9, 'france': 10, 'orderdetails': 11, 'british': 12, 'record': 13, 'technology': 14, 'resolving': 15, 'ibict': 16, 'institute': 17, 'systems': 18, 'agregador': 19, 'repositorio': 20, 'union': 21, 'based': 22, 'show': 23, 'pour': 24, 'fuuml': 25, 'eacute': 26, 'brazil': 27, 'opus': 28, 'teses': 29, 'agrave': 30, 'data': 31, 'analysis': 32, 'networks': 33, 'with': 34, 'atilde': 35, 'reacute': 36, 'dspace': 37, 'application': 38, 'applications': 39, 'technical': 40, 'verlag': 41, 'design': 42, 'learning': 43, 'volltexte': 44, 'cambridge': 45, 'using': 46, 'multi': 47, 'system': 48, 'paris': 49, 'berlin': 50, 'munich': 51, 'london': 52, 'information': 53, 'dans': 54, 'software': 55, 'grenoble': 56, 'deacute': 57, 'massachusetts': 58, 'unicamp': 59, 'college': 60, 'model': 61, 'meacute': 62, 'item': 63, 'reposip': 64, 'models': 65, 'index': 66, 'eprints': 67, 'approach': 68, '

In [270]:
###2.1 get all top words in each sentens
for s in tokenized_list[:5]:
    word_group=[]
    for w in top_words:
        if w in s:
            word_group.append(w)
    print(word_group)

StatementMeta(sparkpool3, 22, 14, Finished, Available)

[]
['germany', 'resolving', 'opus', 'volltexte']
['system', 'california', 'performance', 'support']
['resolving', 'opus', 'networks', 'volltexte', 'mobile', 'problems', 'support', 'towards']
['applications', 'evaluation']

In [271]:
#2.2 get the two words phrase
top_words_temp=top_words.copy()
for s in tokenized_list[:3]:
    word_group=[]
    for w in top_words:
        word_first=''
        if w in s:
            word_first=w
            word_group.append(w)
            word_group.append('-1')
            #print(word_first)
            top_words_temp.remove(w)
            for w1 in top_words_temp:
                if w1 in s:
                    word_group.append(word_first+' ' +w1)
                    word_group.append('-1')
    word_group.append('-2')
    print(word_group)

StatementMeta(sparkpool3, 22, 15, Finished, Available)

['-2']
['germany', '-1', 'germany resolving', '-1', 'germany opus', '-1', 'germany volltexte', '-1', 'resolving', '-1', 'resolving opus', '-1', 'resolving volltexte', '-1', 'opus', '-1', 'opus volltexte', '-1', 'volltexte', '-1', '-2']
['system', '-1', 'system california', '-1', 'system performance', '-1', 'system support', '-1', 'california', '-1', 'california performance', '-1', 'california support', '-1', 'performance', '-1', 'performance support', '-1', 'support', '-1', '-2']

In [282]:
###2.3 output
output=[]
for s in tokenized_list:
    top_words_temp=top_words.copy()
    word_group=[]
    for w in top_words:
        word_first=''
        if w in s:
            word_first=w
            word_group.append(str(word_dict.get(w)))
            word_group.append('-1')
            #print(word_first)
            if w in top_words_temp:
                top_words_temp.remove(w)
            else:
                print(w)
            for w1 in top_words_temp:
                if w1 in s:
                    word_group.append(str(word_dict.get(word_first))+' ' +str(word_dict.get(w1)))
                    word_group.append('-1')
    word_group.append('-2')
    #print(word_group)
    l=''
    for s in word_group:
        l=l+' '+s
    l=l[1:]
    if l!='-2':
        output.append(l)

#print(output)

StatementMeta(sparkpool3, 22, 26, Finished, Available)

In [278]:
print(output)
#[' -2', ' 4 -1 4 15 -1 4 28 -1 4 44 -1 15 -1 15 28 -1 15 44 -1 28 -1 28 44 -1 44 -1 -2', ' 48 -1 48 79 -1 48 91 -1 48 181 -1 79 -1 79 91 -1 79 181 -1 91 -1 91 181 -1 181 -1 -2']

StatementMeta(sparkpool3, 22, 22, Finished, Available)

[' -2', ' 4 -1 4 15 -1 4 28 -1 4 44 -1 15 -1 15 28 -1 15 44 -1 28 -1 28 44 -1 44 -1 -2', ' 48 -1 48 79 -1 48 91 -1 48 181 -1 79 -1 79 91 -1 79 181 -1 91 -1 91 181 -1 181 -1 -2']

In [283]:
output_all = spark.createDataFrame(data=[[p] for p in output],schema =columns)
output_all.coalesce(1).write.format("text").option("header", "false").mode("overwrite").save('adl://cp-bizops-c15.azuredatalakestore.net/local/users/lowen/UIUC/dblp_output.txt')


StatementMeta(sparkpool3, 22, 27, Finished, Available)

In [274]:
tokenized_list[:3]

StatementMeta(sparkpool3, 22, 18, Submitted, Available)

[['carmen', 'heine', 'modell', 'produktion', 'online', 'hilfen', 'aarhus'], ['gerd', 'hoff', 'verfahren', 'thematisch', 'spezialisierten', 'suche', 'seine', 'realisierung', 'prototypen', 'homepagesearch', 'trier', 'germany', 'opus', 'volltexte', 'resolving'], ['margo', 'seltzer', 'file', 'system', 'performance', 'transaction', 'support', 'california', 'berkeley', 'berkeley', 'papers']]

In [275]:
import gensim
from gensim import corpora
from pprint import pprint

# How to create a dictionary from a list of sentences?
documents = ["The Saudis are preparing a report that will acknowledge that", 
             "Saudi journalist Jamal Khashoggi's death was the result of an", 
             "interrogation that went wrong, one that was intended to lead", 
             "to his abduction from Turkey, according to two sources."]

documents_2 = ["One source says the report will likely conclude that", 
                "the operation was carried out without clearance and", 
                "transparency and that those involved will be held", 
                "responsible. One of the sources acknowledged that the", 
                "report is still being prepared and cautioned that", 
                "things could change."]

# Tokenize(split) the sentences into words
texts = [[text for text in doc.split()] for doc in documents]
texts

StatementMeta(sparkpool3, 22, 19, Finished, Available)

[['The', 'Saudis', 'are', 'preparing', 'a', 'report', 'that', 'will', 'acknowledge', 'that'], ['Saudi', 'journalist', 'Jamal', "Khashoggi's", 'death', 'was', 'the', 'result', 'of', 'an'], ['interrogation', 'that', 'went', 'wrong,', 'one', 'that', 'was', 'intended', 'to', 'lead'], ['to', 'his', 'abduction', 'from', 'Turkey,', 'according', 'to', 'two', 'sources.']]

In [276]:
for s in list_dblp_together[:2]:
    s1=word_tokenize(s)
    print(s1)


StatementMeta(sparkpool3, 22, 20, Finished, Available)

NameError: name 'word_tokenize' is not defined

In [ ]:
#import pandas as pd 
#df_dblp=pd.DataFrame.from_dict(list_dblp[:1])
#display(df_dblp)

StatementMeta(, , , Cancelled, )